** Beginner's Guide to Data Mining by Ron Zacharski** <br>
*Chapter 2 - Collabarative Filtering* 

In [1]:
users = {"Angelica":{"Blues Traveler":3.5, "Broken Bells":2.0, "Norah Jones":4.5, "Phoenix":5.0, "Slightly Stoopid":1.5,
                    "The Strokes":2.5, "Vampire Weekend":2.0}, 
        "Bill": {"Blues Traveler": 2.0, "Broken Bells": 3.5, "Deadmau5": 4.0, "Phoenix": 2.0, 
                 "Slightly Stoopid": 3.5, "Vampire Weekend": 3.0},
        "Chan": {"Blues Traveler": 5.0, "Broken Bells": 1.0, "Deadmau5": 1.0, "Norah Jones": 3.0, "Phoenix": 5,
                  "Slightly Stoopid": 1.0},
        "Dan": {"Blues Traveler": 3.0, "Broken Bells": 4.0, "Deadmau5": 4.5, "Phoenix": 3.0, "Slightly Stoopid": 4.5, 
                "The Strokes": 4.0,"Vampire Weekend": 2.0},
        "Hailey": {"Broken Bells": 4.0, "Deadmau5": 1.0, "Norah Jones": 4.0, "The Strokes": 4.0,"Vampire Weekend": 1.0},
        "Jordyn": {"Broken Bells": 4.5, "Deadmau5": 4.0, "Norah Jones": 5.0, "Phoenix": 5.0, "Slightly Stoopid": 4.5,
                   "The Strokes": 4.0, "Vampire Weekend": 4.0},
        "Sam": {"Blues Traveler": 5.0, "Broken Bells": 2.0, "Norah Jones": 3.0, "Phoenix": 5.0, "Slightly Stoopid": 4.0,
                "The Strokes": 5.0},
        "Veronica": {"Blues Traveler": 3.0, "Norah Jones": 5.0, "Phoenix": 4.0, "Slightly Stoopid": 2.5,
                     "The Strokes": 3.0}}

In [2]:
users["Bill"], users["Jordyn"]

({'Blues Traveler': 2.0,
  'Broken Bells': 3.5,
  'Deadmau5': 4.0,
  'Phoenix': 2.0,
  'Slightly Stoopid': 3.5,
  'Vampire Weekend': 3.0},
 {'Broken Bells': 4.5,
  'Deadmau5': 4.0,
  'Norah Jones': 5.0,
  'Phoenix': 5.0,
  'Slightly Stoopid': 4.5,
  'The Strokes': 4.0,
  'Vampire Weekend': 4.0})

In [44]:
users["Veronica"]["Blues Traveler"], users["Dan"]["Deadmau5"]

(3.0, 4.5)

The Manhattan distance is simply the distance between two points in a N dimensional space based on a horizontal or vertical path between the points.
In our scenario, the Manhattan distance between two users will simply be the sum of the absolute differences between the ratings of user 1 and user 2 for each band rated by both the users.

Code to compute the Manhattan distance

In [4]:
def Manhattan(rating1, rating2):
    ### Computes Manhattan distance between two users ###
    distance = 0
    for key in rating1:
        if key in rating2:
            distance += (abs(rating1[key] - rating2[key]))
    return distance

In [5]:
Manhattan(users["Veronica"], users["Hailey"])

2.0

In [6]:
Manhattan(users["Angelica"], users["Chan"])

4.5

Now that we know how to compute Manhattan distances, we can find out the nearest neighbors to our user of interest based on their Manhattan distances from our user. 

Function to find the nearest person

In [7]:
def nearestneighbor(username, users):
    ###creates a sorted list of users based on their Manhattan distance to the username###
    distances = []
    for user in users:
        if user != username:
            distance = Manhattan(users[user], users[username])
            distances.append((distance, user))
    distances.sort()
    return distances

In [8]:
print(nearestneighbor("Hailey", users))

[(2.0, 'Veronica'), (4.0, 'Chan'), (4.0, 'Sam'), (4.5, 'Dan'), (5.0, 'Angelica'), (5.5, 'Bill'), (7.5, 'Jordyn')]


In [9]:
users[nearestneighbor("Hailey", users)[0][1]]["Phoenix"]

4.0

We now have enough tools to start making recommendations to our users. We can accomplish this in a few steps.
First, find out the nearest neighbor to our user using the nearestneighbor function. Next, get the ratings of all the bands which our neighbor has rated but the user has not. From these set of bands, we recommend to our user those with the highest ratings.

Now let's create a function which provides recommendations to users

In [10]:
def recommend(username, users):
    ### find the nearest neighbor to our username
    nearest = nearestneighbor(username, users)[0][1]
    recommendations = []
    ### get the list of bands which the username rated and the list of bands the nearest neighbor rated
    neighborRatings = users[nearest]
    usernameRatings = users[username]
    ### create a sorted list of bands which the neighbor has rated but username has not
    for artist in neighborRatings:
        if artist not in usernameRatings:
            recommendations.append((neighborRatings[artist], artist))
    recommendations.sort(reverse=True)
    return recommendations

In [11]:
recommend("Hailey", users)

[(4.0, 'Phoenix'), (3.0, 'Blues Traveler'), (2.5, 'Slightly Stoopid')]

This is a very simple and basic recommender. As one can clearly see, there are a few flaws with it. For instance, if our neighbor has only rated bands rated by our user, we do not have any recommendations to make based on the ratings of our nearest neighbor. This is because we do not have a single band which is rated by or neighbor but not by our user.

Below is an example.

In [12]:
recommend("Angelica", users)

[]

This shows us that using just one neighbor is not a great idea to make recommendations.

The Manhattan distance is not the only way to calculate distances between two users. This measure comes from a more general function called the Minkowski distance function. The Minkowski distance function is shown below.

$$d(x, y) = (\sum_{k=1}^{k=n} |x_{k} - y_{k}|^r)^{(1/r)}$$

For r = 1, we get the Manhattan distance.<br>
For r = 2, we get the Euclidean distance.

Now, it is time to make our distance function more flexible. We will modify our Manhattan function and generalize it to calculate Minkowski distance.

In [13]:
def Minkowski(rating1, rating2, r):
    ###determines the Minkowski distance between two users###
    distance = 0
    for band in rating1:
        if band in rating2:
            distance += (abs(rating1[band] - rating2[band]))**r
    return (distance)**(1/r)

We will now alter the nearest neighbor function to use the Minkowski distance. This function creates a sorted list of users based on their Minkowski distance with a particular user of interest.

In [14]:
from operator import itemgetter

In [15]:
def nearestneighbor(username, users, r):
    ###sorted list of users based on their distance to username###
    distances = []
    for user in users:
        if user != username:
            distance = Minkowski(users[user], users[username], r)
            distances.append((user, distance))
    return sorted(distances, key = itemgetter(1))

In [16]:
Minkowski(users["Hailey"], users["Jordyn"], 2)

4.387482193696061

In [17]:
nearestneighbor("Dan", users, 2)

[('Bill', 2.1213203435596424),
 ('Veronica', 2.449489742783178),
 ('Jordyn', 2.9154759474226504),
 ('Hailey', 3.640054944640259),
 ('Sam', 3.640054944640259),
 ('Angelica', 4.415880433163924),
 ('Chan', 6.442049363362563)]

In [18]:
users["Dan"]["Broken Bells"]

4.0

The Minkowski distance function also has its own flaws. One of them is that users have different ways to rate. Some people are very generous with their ratings and will never rate lower than a 3 (on a 5 point scale). On the other hand, some users are conservative and would never rate anything a 4 or a 5 (again, on a scale on 1 to 5) even if everything is perfect. This subjectivity can affect our Minkowski distance metric and, ultimately, our recommendations.

Fortunately, there is a way around this. We have another metric called the Pearson Correlation which calculates the similarity between users based on their ratings. This valus always lies between -1 and 1, inclusive. This allows us to understand the similarity between user ratings on an objective scale, even if their subjective scales were completely different. <br>
A value of 1 implies that our users are perfectly similar in their ratings whereas a value of -1 implies perfect dissimilarity.

In the book, the author provides an approximation of the pearson correlation coefficient. This is just a matter of convenience as it allows us to calculate the value with just a single pass of the dataset.<br>
This approximation is given below.

$$r = \frac {\sum_{i=1}^{i=n} x_{i}y_{i} - \frac{\sum_{i=1}^{i=n} x_{i}\sum_{i=1}^{i=n}y_{i}}{n}}{\sqrt{\sum_{i=1}^{i=n} x^2_{i} - \frac{{(\sum_{i=1}^{i=n} x_{i})}^{2}}{n}} \sqrt{\sum_{i=1}^{i=n} y^2_{i} - \frac{{(\sum_{i=1}^{i=n} y_{i})}^{2}}{n}}} $$

Let us implement the approximation of the pearson correlation coefficient in the book

In [19]:
import pandas as pd
from math import isnan, sqrt

In [20]:
def pearson(user1, user2):
    ### calculates the pearson correlation coefficient approximation ###
    user1_total = 0
    user1_sq = 0
    user2_total = 0
    user2_sq = 0
    product = 0
    n = 0
    for band in user1:
        if band in user2:
            user1_total += user1[band]
            user2_total += user2[band]
            user1_sq += user1[band]**2
            user2_sq += user2[band]**2
            product += user1[band]*user2[band]
            n += 1
    ### return 0 if no common bands ###
    if n == 0:
        return 0
    ### return 0 if denominator is zero ###
    elif (sqrt(user1_sq - (user1_total**2/n)==0)) or (sqrt(user2_sq - (user2_total**2/n))==0):                                               
        return 0
    else:
        value = (product - (user1_total*user2_total/n))/((sqrt(user1_sq - (user1_total**2/n)))*(sqrt(user2_sq - (user2_total**2/n))))
    return value

In [21]:
pearson(users["Angelica"], users["Bill"])

-0.9040534990682699

In [22]:
pearson(users["Angelica"], users["Hailey"])

0.42008402520840293

In [23]:
pearson(users["Angelica"], users["Jordyn"])

0.7639748605475432

Before we start making a recommender system, let us make one more function to measure similarity between users using the Cosine Similarity metric. This metric is extremely useful when we have sparse data.

In [24]:
def cosine(user1, user2):
    sum1 = 0
    sum2 = 0
    product = 0
    n = 0
    for movie in user1:
        sum1 += user1[movie]**2
        if movie in user2:
            product += user1[movie]*user2[movie]
            n += 1
    for movie in user2:
        sum2 += user2[movie]**2
    if n == 0:
        return 0
    else:
        return product/(sqrt(sum1)*sqrt(sum2))

In [66]:
cosine(users["Angelica"], users["Veronica"])

0.9246279432210068

In [67]:
cosine(users["Angelica"], users["Jordyn"])

0.8025694479618785

In [68]:
cosine(users["Angelica"], users["Bill"])

0.540392532615596

Let us put all this information to user and make a simple recommender system. We have data about 25 users and their ratings for 25 movies. We will user collabarative filtering to make our recommendations.

In [25]:
ratings = pd.read_csv('Movie_Ratings.csv', index_col=0)

In [26]:
ratings.head()

,Patrick C,Heather,Bryan,Patrick T,Thomas,aaron,vanessa,greg,brian,ben,...,Zak,Matt,Chris.1,Josh,Amy,Valerie,Gary,Stephen,Jessica,Jeff
Alien,NaN,NaN,2.0,NaN,5.0,4.0,NaN,NaN,4.0,NaN,...,NaN,NaN,4.0,3.0,NaN,NaN,2.0,5.0,NaN,4.0
Avatar,4.0,5.0,5.0,4.0,2.0,NaN,4.0,3.0,NaN,3.0,...,5.0,NaN,NaN,4.0,3.0,2.0,1.0,4.0,NaN,4.0
Blade Runner,5.0,NaN,NaN,NaN,5.0,4.0,NaN,1.0,5.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,1.0,NaN,NaN,5.0
Braveheart,4.0,NaN,5.0,NaN,4.0,4.0,3.0,4.0,4.0,NaN,...,5.0,NaN,4.0,NaN,3.0,4.0,5.0,5.0,NaN,4.0
Dodgeball,5.0,4.0,3.0,2.0,4.0,NaN,4.0,5.0,3.0,4.0,...,3.0,NaN,3.0,NaN,4.0,3.0,4.0,3.0,NaN,3.0


In [27]:
ratings = ratings.to_dict('dict')

In [28]:
type(ratings)

dict

We have some nan values in our dictionary representing movies which have not been rated by users. We will remove these values so that our dictionary does not have any nan values. This helps us to use our Minkowski, nearestneighbor, and other fucntions without having to make any modifications to them.

In [29]:
### remove nans from the dictionary
clean_ratings = {k1:{k2:v2 for k2, v2 in v1.items() if (isnan(v2) == False)} for k1, v1 in ratings.items()}

In [30]:
Manhattan(clean_ratings["Heather"], clean_ratings["Bryan"])

12.0

In [31]:
Manhattan(clean_ratings["brian"], clean_ratings["ben"])

11.0

In [32]:
Minkowski(clean_ratings["vanessa"], clean_ratings["greg"], 2)

4.0

In [33]:
nearestneighbor("Matt", clean_ratings, 2)

[('Josh', 1.0),
 ('vanessa', 1.4142135623730951),
 ('Amy', 1.4142135623730951),
 ('ben', 2.4494897427831779),
 ('Zwe', 2.4494897427831779),
 ('Jeff', 2.4494897427831779),
 ('Patrick T', 2.6457513110645907),
 ('Gary', 2.6457513110645907),
 ('Jessica', 2.6457513110645907),
 ('greg', 2.8284271247461903),
 ('Bryan', 3.3166247903553998),
 ('Jonathan', 3.3166247903553998),
 ('Heather', 3.4641016151377544),
 ('Erin', 3.4641016151377544),
 ('Zak', 3.7416573867739413),
 ('Chris', 3.872983346207417),
 ('Katherine', 4.0),
 ('Thomas', 4.2426406871192848),
 ('Patrick C', 4.358898943540674),
 ('Chris.1', 4.358898943540674),
 ('aaron', 4.5825756949558398),
 ('brian', 4.8989794855663558),
 ('Valerie', 5.0990195135927845),
 ('Stephen', 5.0990195135927845)]

**Building the recommender system**

We will go through the following steps to build our recommender system.

First, we find out the *k nearest neighbors* to our user of interest using the nearest neighbor function. We then calculate the cosine similarity of each neighbor with our user. This will serve to determine the order of similarity between the user and his/her neighbors. <BR>
Now, for every neighbor, we find out movies which were not rated by our user, and multiply the movie's rating and the weight of the cosine similarity of that neighbor, to get the recommended rating for that movie. We sum the recommended ratings for each user and get the final rating for each movie. 

We then recommend the top 3 rated movies to our user.

In [36]:
def recommender(our_user, users, k):
    ### Getting the k nearest neigbors for our user
    persons = []
    for i in range(k):
        persons.append(nearestneighbor(our_user, users, 2)[i][0])
    ### Get the cosine similarity of our k nearest neighbors
    weights = []
    for j in persons:
        weights.append(cosine(users[our_user], users[j]))
    recommendations = {}
    ### Multiply the rating of the movie with weight of the cosine similarity of the neighbor, for each neighbor
    ### Add this product for every movie to get the recommended rating for that movie
    for m in range(k):
        for movie in users[persons[m]]:
            if not movie in users[our_user]:
                if not movie in recommendations:
                    recommendations[movie] = users[persons[m]][movie]*(weights[m]/sum(weights))
                else:
                    recommendations[movie] += users[persons[m]][movie]*(weights[m]/sum(weights))
    ### Sort the list and recommend the top 3 movies
    final_list = list(recommendations.items())
    final_list.sort(key = lambda x:x[1], reverse=True)
    if len(final_list) < 3:
        return [l[0] for l in final_list]
    else:
        return [l[0] for l in final_list][0:3]

In [37]:
recommender("vanessa", clean_ratings, 3)

['Spiderman', 'The Dark Knight', 'Blade Runner']

In [38]:
recommender("Gary", clean_ratings, 3)

['The Dark Knight']

In [39]:
recommender("Zak", clean_ratings, 3)

['Jaws', 'Alien']